<a href="https://colab.research.google.com/github/NaviaEddy/PerceiverIO_SIS421/blob/main/PerceiverIO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PerceiverIO

In [1]:
import requests
import torch
from PIL import Image
from transformers import pipeline

In [3]:
!pip install perceiver-io[text]
!pip install perceiver-io[vision]

In [14]:
#TEXTO
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import AutoModelForMaskedLM, AutoTokenizer
from perceiver.model.text import mlm
from perceiver.model.text import classifier

#IMAGENES - VIDEOS
from transformers import pipeline
from perceiver.data.vision import video_utils
from perceiver.model.vision import optical_flow
from transformers import AutoModelForImageClassification, AutoImageProcessor
from perceiver.model.vision import image_classifier

# **Analisis de sentimientos**

In [6]:
repo_id = "krasserm/perceiver-io-txt-clf-imdb"
sentiment_pipeline = pipeline("sentiment-analysis", model=repo_id)

In [7]:
mini_reviews = [
    "I've seen this movie yesterday and it was really boring.",
    "I can recommend this movie to all fantasy movie lovers.",
]
predictions = sentiment_pipeline(mini_reviews)

for r, p in zip(mini_reviews, predictions):
    print(f"{r} ({p['label']})")

I've seen this movie yesterday and it was really boring. (NEGATIVE)
I can recommend this movie to all fantasy movie lovers. (POSITIVE)


# **Prediccion de palabras**

# **Flujo Optimo de imagenes**

In [8]:
optical_flow_pipeline = pipeline("optical-flow", model="krasserm/perceiver-io-optical-flow", device="cuda:0")

config.json:   0%|          | 0.00/1.55k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/353M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [9]:
frame_1 = Image.open(requests.get("https://martin-krasser.com/perceiver/flow/frame_0047.png", stream=True).raw)
frame_2 = Image.open(requests.get("https://martin-krasser.com/perceiver/flow/frame_0048.png", stream=True).raw)

rendered_optical_flow = optical_flow_pipeline((frame_1, frame_2), render=True)

Image.fromarray(rendered_optical_flow).save("optical_flow.png")

# **Flujo Optimo de videos**

In [11]:
optical_flow_pipeline = pipeline("optical-flow", model="krasserm/perceiver-io-optical-flow", device="cuda:0")

In [12]:
# sample consecutive video frame pairs
frame_pairs = video_utils.read_video_frame_pairs("/content/206294_medium.mp4")

# create and render optical flow for all frame pairs
optical_flows = optical_flow_pipeline(frame_pairs, render=True, device="cuda:0")

# create video with rendered optical flows
video_utils.write_video("caballos_output.mp4", optical_flows, fps=24)

## Clasificación de imágenes

In [15]:
repo_id = "krasserm/perceiver-io-img-clf"

model = AutoModelForImageClassification.from_pretrained(repo_id)
processor = AutoImageProcessor.from_pretrained(repo_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/70.7k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/246M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/551 [00:00<?, ?B/s]